In [1]:
import torch

1. `*` (elementwise multiply)

* What it does: multiplies tensors elementwise (same shape after broadcasting). Supports broadcasting and dtype promotion; equivalent to `torch.mul(t1, t2)`. ([PyTorch Docs][1])
* Important args/notes: no explicit args (uses Tensor `__mul__`); broadcasting rules apply; result participates in autograd like any tensor op.
* Example:

In [6]:
a = torch.tensor([1.,2.,3.])
b = torch.tensor([10.,20.,30.])
c = a * b   # tensor([10., 40., 90.])
# broadcasting:
x = torch.randn(2,3); s = torch.tensor([2.,3.,4.])
y = x * s   # s broadcast across dim0

w = torch.tensor([2,3, 4])
z = w * s

2. `torch.mul(input, other, *, out=None)` / `Tensor.mul` / `Tensor.mul_`

* What it does: elementwise multiply (same semantics as `*`). `torch.mul` is the functional form; `Tensor.mul_` is the **in-place** variant that overwrites the tensor. `torch.mul` supports an `out=` keyword to write result into a provided tensor. ([PyTorch Docs][1])
* Important args/notes:

  * `other` can be scalar or tensor (broadcasting applies).
  * `out` (optional): preallocated tensor to receive the result (avoids allocation).
  * In-place `mul_()` modifies the receiver (breaks views/grad expectations if used carelessly).
* Example:

In [8]:
z = torch.empty(3)
torch.mul(a, b, out=z)   # writes into z
a.mul_(2)                # a modified in-place

tensor([2., 4., 6.])

3. `@` (Python matrix-multiply operator → routes to `matmul`) / `torch.matmul(input, other, *, out=None)`

* What it does: linear-algebra product with behavior depending on rank:

  * 1D @ 1D → dot (scalar).
  * 2D @ 2D → matrix×matrix.
  * ND @ ND (N≥3) → batched matmul with broadcasting of batch dims. Note: the 1D dot case may not support `out=` in some versions. ([PyTorch Docs][2])
* Important args/notes:

  * `out` (optional) for most cases (check docs: 1D·1D may not accept `out`).
  * Broadcasting rules for batch dims; shapes must be compatible for matrix multiplication on the last two dims.
* Example:

In [25]:
# 1D tensor
D = torch.tensor([3., 4.])

# 2D matmul
A = torch.randn(2,3); B = torch.randn(3,4)
C = A @ B   # (2,4)

# batched matmul
X = torch.randn(5,2,3); W = torch.randn(5,3,4)
Y = torch.matmul(X, W)   # (5,2,4)

4. `torch.mm(input, mat2, *, out=None, out_dtype=None)` (strict 2D matrix multiply)

* What it does: matrix multiplication **only** for 2-D tensors (no batching, no broadcasting across batch dims). Use for plain 2D matrix×matrix when you want the strict 2D contract. ([PyTorch Docs][3])
* Important args/notes:

  * Both inputs must be 2D.
  * `out` optional. Some GPU backends expose `out_dtype` controls for precision on CUDA/ROCm.
* Example:

In [29]:
A = torch.randn(2,3); B = torch.randn(3,4)
R = torch.mm(A, B)   # (2,4)

5. `torch.bmm(input, mat2, *, out=None, out_dtype=None)` (batched matrix multiply)

* What it does: batched 2-D matmul for **3-D** tensors: `(B, N, M) @ (B, M, P) -> (B, N, P)`. No broadcasting over the batch dimension — batch sizes must match. ([PyTorch Docs][4])
* Important args/notes:

  * Inputs must be 3D and share the same batch size.
  * `out` optional; `out_dtype` available on some devices.
* Example:

In [33]:
A = torch.randn(10,2,3); B = torch.randn(10,3,4)
C = torch.bmm(A, B)   # (10,2,4)

6. `torch.einsum(equation, *operands, /, *, out=None)`

* What it does: general Einstein summation—specify index labels to compute arbitrary contractions (matmul, batched matmul, trace, outer, inner, transpositions, etc.) by an equation string like `"ij,jk->ik"`. Extremely flexible and expressive. ([PyTorch Docs][5])
* Important args/notes:

  * `equation` is a string describing indices; operands must match the labels.
  * `out` optional.
  * Can express operations that would otherwise require combinations of permute/reshape/matmul; but can be slower for some simple ops—benchmark if performance-critical.
* Example:

In [ ]:
# matmul
C = torch.einsum("ij,jk->ik", A, B)

# batched matmul
Y = torch.einsum("bij,bjk->bik", X, W)

7. `torch.dot(input, tensor, *, out=None)`

* What it does: dot product of **two 1-D** tensors only (strict 1D contract). Unlike NumPy, `torch.dot` intentionally requires both inputs be 1-D and same length. ([PyTorch Docs][6])
* Important args/notes:

  * Inputs must be 1D and same size.
  * `out` optional. For batched or higher-dim reductions, use `torch.inner`, `torch.matmul`, or `torch.linalg.vecdot`.
* Example:

In [ ]:
u = torch.tensor([1.,2.,3.]); v = torch.tensor([10.,20.,30.])
s = torch.dot(u, v)   # scalar 140.

8. `torch.inner(input, other, *, out=None)`

* What it does: computes dot over the **last dimension**: for 1-D tensors it equals `dot`; for higher dims it sums products over the last axis of each tensor and broadcasts over leading axes. Useful for batched inner products. ([PyTorch Docs][7])
* Important args/notes:

  * If either input is scalar, behaves like elementwise multiply.
  * Supports broadcasting on leading dims; result shape equals broadcasted leading dims.
* Example:

In [ ]:
# 1D
torch.inner(u, v)   # scalar

# batched
x = torch.randn(4,3); y = torch.randn(4,3)
r = torch.inner(x, y)   # (4,)

9. `torch.vdot(input, other, *, out=None)`

* What it does: dot product that **conjugates the first argument** (for complex tensors) and then sums; flattens inputs to 1D first. For real tensors behaves like `dot`. Use when working with complex vectors and you need conjugation semantics. ([PyTorch Docs][8])
* Important args/notes:

  * Inputs are flattened; both should be 1-D-equivalent length.
  * Conjugates `input` if complex (`conj(input) * other` summed).
  * For batched vector-dot use `torch.linalg.vecdot` (different API).
* Example:

In [ ]:
a = torch.tensor([1+2j, 3+4j])
b = torch.tensor([10+0j, 20+0j])
s = torch.vdot(a, b)   # uses conj(a)

10. `torch.outer(input, other)` and `torch.kron` (optional helpers often used) — brief mention

* `torch.outer` computes the outer product of 1-D vectors → matrix; `torch.kron` computes the Kronecker product. Use these when you need expansions rather than contractions. (Docs: general torch reference.) ([PyTorch Docs][9])
* Example:

In [ ]:
outer = torch.outer(u, v)   # shape (3,3)
k = torch.kron(torch.tensor([[1,2]]), torch.tensor([[3,4]]))

— end.

[1]: https://docs.pytorch.org/docs/stable/generated/torch.mul.html?utm_source=chatgpt.com "torch.mul — PyTorch 2.10 documentation"
[2]: https://docs.pytorch.org/docs/stable/generated/torch.matmul.html?utm_source=chatgpt.com "torch.matmul — PyTorch 2.10 documentation"
[3]: https://docs.pytorch.org/docs/stable/generated/torch.mm.html?utm_source=chatgpt.com "torch.mm — PyTorch 2.10 documentation"
[4]: https://docs.pytorch.org/docs/stable/generated/torch.bmm.html?utm_source=chatgpt.com "torch.bmm — PyTorch 2.10 documentation"
[5]: https://docs.pytorch.org/docs/stable/generated/torch.einsum.html?utm_source=chatgpt.com "torch.einsum — PyTorch 2.10 documentation"
[6]: https://docs.pytorch.org/docs/stable/generated/torch.dot.html?utm_source=chatgpt.com "torch.dot — PyTorch 2.10 documentation"
[7]: https://docs.pytorch.org/docs/stable/generated/torch.inner.html?utm_source=chatgpt.com "torch.inner — PyTorch 2.10 documentation"
[8]: https://docs.pytorch.org/docs/stable/generated/torch.vdot.html?utm_source=chatgpt.com "torch.vdot — PyTorch 2.10 documentation"
[9]: https://docs.pytorch.org/docs/stable/torch.html?utm_source=chatgpt.com "torch — PyTorch 2.9 documentation"